<a href="https://colab.research.google.com/github/LShahmiri/Python_Projects/blob/master/Export_MI_to_Excel/Export_MI_to_Excel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install the required library to write data to Excel files
!pip install xlsxwriter

import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import xlsxwriter
from skimage.io import imread
from skimage.exposure import histogram
from skimage.color import rgb2gray
import glob

def mutual_information(histogram):
    """
    Calculate mutual information from a 2D histogram.

    Args:
        histogram (np.ndarray): 2D histogram.

    Returns:
        float: Mutual information value.
    """
    pxy = histogram / float(np.sum(histogram))
    px = np.sum(pxy, axis=1)  # marginal for x over y
    py = np.sum(pxy, axis=0)  # marginal for y over x
    px_py = px[:, None] * py[None, :]  # Broadcast to multiply marginals
    nzs = pxy > 0  # Only non-zero pxy values contribute to the sum
    return np.sum(pxy[nzs] * np.log(pxy[nzs] / px_py[nzs]))

def calculate_mutual_information(image1, image2):
    """
    Calculate mutual information between two images.

    Args:
        image1 (np.ndarray): First image.
        image2 (np.ndarray): Second image.

    Returns:
        float: Mutual information value.
    """
    hist_2d, _, _ = histogram(image1.ravel(), image2.ravel(), bins=20)
    return mutual_information(hist_2d)

def process_images_in_folder(folder_path, worksheet):
    """
    Process images in a folder and write mutual information values to the worksheet.

    Args:
        folder_path (str): Path to the folder containing images.
        worksheet (xlsxwriter.worksheet.Worksheet): Excel worksheet object.
    """
    for file_path in glob.glob(os.path.join(folder_path, '*.*')):
        filename = os.path.basename(file_path)
        image1 = cv2.imread(os.path.join(folder_path, '1.JPG'))
        image2 = cv2.imread(file_path)

        correlation_coefficient = np.corrcoef(image1.ravel(), image2.ravel())[0, 1]
        mutual_info = calculate_mutual_information(image1, image2)

        worksheet.write('A' + str(i), mutual_info)
        worksheet.write('B' + str(i), filename)
        i += 1

def main():
    # Specify the path to the Excel file to be created
    excel_file_path = '/content/drive/MyDrive/excel/100-chines.xlsx'

    # Specify the root directory containing subdirectories with image files
    root_directory = '/content/drive/MyDrive/excel/pooshe/'

    # Create a new Excel workbook and add a worksheet
    workbook = xlsxwriter.Workbook(excel_file_path)
    worksheet = workbook.add_worksheet()

    # Set image plotting configurations
    plt.rcParams['image.cmap'] = 'BrBG'
    plt.rcParams['image.interpolation'] = 'nearest'

    i = 1
    for folder_name in os.listdir(root_directory):
        folder_path = os.path.join(root_directory, folder_name)

        # Create a new worksheet for each subdirectory (folder)
        worksheet_name = str(folder_name)
        worksheet = workbook.add_worksheet(worksheet_name)

        # Process images in the subdirectory and write mutual information values
        process_images_in_folder(folder_path, worksheet)

    # Close the workbook to save the changes
    workbook.close()

if __name__ == "__main__":
    main()
